<a href="https://colab.research.google.com/github/HanhaodiZhang/HATE/blob/main/emojiclean_12162020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qq transformers
!pip install twokenize

In [2]:
import twokenize
twokenize.tokenize("lol ly x0x0,:D")

['lol', 'ly', 'x0x0', ',', ':D']

In [3]:
import torch
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv("olid-training.tsv", sep='\t', header=0, index_col='id')

df.head()

,tweet,subtask_a,subtask_b,subtask_c
id,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


In [5]:
possible_labels=df.subtask_a.unique()
label_dict={}
for index, possible_label in enumerate(possible_labels):
  label_dict[possible_label]=index

label_dict

{'NOT': 1, 'OFF': 0}

In [6]:
df['a_label']=df.subtask_a.replace(label_dict)
df.head()
df.head()

,tweet,subtask_a,subtask_b,subtask_c,a_label
id,,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN,0
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,0
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN,1
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN,0
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN,1


In [7]:
!pip install emoji --upgrade

Requirement already up-to-date: emoji in /usr/local/lib/python3.6/dist-packages (0.6.0)


In [8]:
!pip install wordsegment

In [9]:
import emoji
import re
def foremoji(text):
  text=emoji.demojize(text)
  text = text.split()
  new_text=[]
  for w in text:
    w=re.sub(r":.", "<emoji>", w)
    w=re.sub(r".:", "<emoji>", w)
    w=re.sub(r"_"," ",w)
    new_text.append(w)
  text = " ".join(new_text)
  return text

In [10]:
from wordsegment import load
load()

In [11]:
from wordsegment import segment
import emoji
import re
def foremoji(text):
  text=emoji.demojize(text)
  text = text.split()
  new_text=[]
  for w in text:
    w=re.sub(r":.", "<emoji>", w)
    w=re.sub(r".:", "<emoji>", w)
    w=re.sub(r"_", " ", w)
    new_text.append(w)
  text = " ".join(new_text)
  return text

In [12]:
df['emoji_tweet']=df.tweet.apply(foremoji)

In [13]:
df.head()

,tweet,subtask_a,subtask_b,subtask_c,a_label,emoji_tweet
id,,,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN,0,@USER She should ask a few native Americans wh...
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,0,@USER @USER Go home you’re drunk!!! @USER #MAG...
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN,1,Amazon is investigating Chinese employees who ...
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN,0,"@USER Someone should'veTaken"" this piece of sh..."
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN,1,@USER @USER Obama wanted liberals &amp; illega...


In [14]:
from sklearn.model_selection import train_test_split
RANDOM_SEED=17

X_train,X_val,y_train,y_val = train_test_split(
  df.index.values,
  df.a_label.values,
  test_size=0.2,
  random_state=RANDOM_SEED,
  stratify=df.a_label.values
)

In [15]:
df['data_type']=['not_set']*df.shape[0]

df.loc[X_train,'data_type']='train'
df.loc[X_val,'data_type']='val'
df.head()

,tweet,subtask_a,subtask_b,subtask_c,a_label,emoji_tweet,data_type
id,,,,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN,0,@USER She should ask a few native Americans wh...,train
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,0,@USER @USER Go home you’re drunk!!! @USER #MAG...,val
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN,1,Amazon is investigating Chinese employees who ...,train
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN,0,"@USER Someone should'veTaken"" this piece of sh...",train
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN,1,@USER @USER Obama wanted liberals &amp; illega...,train


In [16]:
from transformers import BertTokenizer

PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [17]:

from transformers import BertForSequenceClassification

model=BertForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME,
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states=False,
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [18]:
tokenizer.add_special_tokens({'additional_special_tokens':["<emoji>"]})
model.resize_token_embeddings(len(tokenizer))

Embedding(28997, 768)

In [19]:



encoder_data_train=tokenizer.batch_encode_plus(
    df[df.data_type=='train'].emoji_tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)

encoder_data_val=tokenizer.batch_encode_plus(
    df[df.data_type=='val'].emoji_tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation =True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)


In [20]:
inputs_ids_train=encoder_data_train['input_ids']
attentions_masks_train=encoder_data_train['attention_mask']
labels_train=torch.tensor(df[df.data_type=='train'].a_label.values)

inputs_ids_val=encoder_data_val['input_ids']
attentions_masks_val=encoder_data_val['attention_mask']
labels_val=torch.tensor(df[df.data_type=='val'].a_label.values)

In [21]:

from torch.utils.data import TensorDataset
dataset_train=TensorDataset(inputs_ids_train,
              attentions_masks_train, labels_train)
dataset_val=TensorDataset(inputs_ids_val,
              attentions_masks_val, labels_val)

In [22]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

BATCH_SIZE=16

dataloader_train=DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=BATCH_SIZE
)
dataloader_val=DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=BATCH_SIZE
)

In [23]:

from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
epochs = 4

scheduler = get_linear_schedule_with_warmup(optimizer, 
                        num_warmup_steps=0,
                        num_training_steps=len(dataloader_train)*epochs)

In [24]:
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def classification_report_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return classification_report(labels_flat, preds_flat, digits=5)

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [25]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':batch[0],
              'attention_mask':batch[1],
              'labels':batch[2],
              }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [26]:
epochs=3

In [27]:
from tqdm.notebook import tqdm
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)             
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.4950102179162092
Validation loss: 0.43880832998687963
F1 Score (Weighted): 0.7880868325715285



Epoch 2
Training loss: 0.37914229990222303
Validation loss: 0.4442911449685154
F1 Score (Weighted): 0.7890490987507243



Epoch 3
Training loss: 0.29856984140041914
Validation loss: 0.5280265356582331
F1 Score (Weighted): 0.7879221275883128



In [28]:
df_sweet = pd.read_csv("testset-levela.tsv", sep='\t', header=0, index_col='id')
df_sweet.head()

,tweet
id,
15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...
27014,"#ConstitutionDay is revered by Conservatives, ..."
30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...
13876,#Watching #Boomer getting the news that she is...
60133,#NoPasaran: Unity demo to oppose the far-right...


In [29]:
df_l = pd.read_csv("labels-levela.csv",header=None,names=['id','subtask_a'],index_col='id')
df_l.head()

,subtask_a
id,
15923,OFF
27014,NOT
30530,NOT
13876,NOT
60133,OFF


In [30]:
test = pd.merge(df_sweet, df_l, on=['id'])

In [31]:
possible_labels=test.subtask_a.unique()
label_dict={}
for index, possible_label in enumerate(possible_labels):
  label_dict[possible_label]=index

label_dict

{'NOT': 1, 'OFF': 0}

In [32]:
test['a_label']=test.subtask_a.replace(label_dict)
test.head()

,tweet,subtask_a,a_label
id,,,
15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF,0
27014,"#ConstitutionDay is revered by Conservatives, ...",NOT,1
30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,NOT,1
13876,#Watching #Boomer getting the news that she is...,NOT,1
60133,#NoPasaran: Unity demo to oppose the far-right...,OFF,0


In [33]:
test['data_type']=['test']*test.shape[0]

In [34]:
test.head()

,tweet,subtask_a,a_label,data_type
id,,,,
15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF,0,test
27014,"#ConstitutionDay is revered by Conservatives, ...",NOT,1,test
30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,NOT,1,test
13876,#Watching #Boomer getting the news that she is...,NOT,1,test
60133,#NoPasaran: Unity demo to oppose the far-right...,OFF,0,test


In [35]:
test['emoji_tweet']=test.tweet.apply(foremoji)

In [36]:
test.head()

,tweet,subtask_a,a_label,data_type,emoji_tweet
id,,,,,
15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF,0,test,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...
27014,"#ConstitutionDay is revered by Conservatives, ...",NOT,1,test,"#ConstitutionDay is revered by Conservatives, ..."
30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,NOT,1,test,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...
13876,#Watching #Boomer getting the news that she is...,NOT,1,test,#Watching #Boomer getting the news that she is...
60133,#NoPasaran: Unity demo to oppose the far-right...,OFF,0,test,#NoPasara<emoji> Unity demo to oppose the far-...


In [37]:

encoder_data_test=tokenizer.batch_encode_plus(
    test[test.data_type=='test'].emoji_tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)

In [38]:
inputs_ids_test=encoder_data_test['input_ids']
attentions_masks_test=encoder_data_test['attention_mask']
labels_test=torch.tensor(test[test.data_type=='test'].a_label.values)
dataset_test=TensorDataset(inputs_ids_test,
              attentions_masks_test, labels_test)
dataloader_test=DataLoader(
    dataset_test,
    sampler=RandomSampler(dataset_test),
    batch_size=4
)

In [47]:

model.load_state_dict(torch.load('finetuned_BERT_epoch_3.model', map_location=torch.device('cuda')))

<All keys matched successfully>

In [48]:
_, predictions, true_test = evaluate(dataloader_test)

In [49]:
accuracy_per_class(predictions, true_test)

Class: OFF
Accuracy: 158/240

Class: NOT
Accuracy: 566/620



In [50]:
print(classification_report_func(predictions, true_test))

              precision    recall  f1-score   support

           0    0.74528   0.65833   0.69912       240
           1    0.87346   0.91290   0.89274       620

    accuracy                        0.84186       860
   macro avg    0.80937   0.78562   0.79593       860
weighted avg    0.83769   0.84186   0.83871       860

